<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/03_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# トピックモデリング (topic modeling)

* BoW (bag-of-words) の範囲で実現できる優れたEDA (exploratory data analysis)。


## 解説

### 使いみち
* テキストの集合から、多数の異なる話題を、それぞれの話題を端的に表す単語リストとして取り出せる。
* 単語の出現頻度を要素とするベクトルの次元圧縮にも使えるが・・・
 * トピックモデルは、次元圧縮の手法としての性能はあまり良くない。
* EDAの手法として使うのが吉。

### 入力データの形式
* 入力データは各文書における各単語の出現回数。
* BoWとしてテキストをモデリングするので、**語順は考慮されない**。

### 代表的な手法: 潜在的ディリクレ配分法 (LDA; latent Dirichlet allocation)
* LDAはテキスト集合のモデリングに使えるベイズ的な確率モデル。
 * LDAの理屈については「統計モデリング2」で。
* 今回はsklearnの実装を使う。
 * https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html


### gensimのLDAは非推奨
* https://radimrehurek.com/gensim/models/ldamodel.html
* 最大の理由: デフォルトの設定で`passes=1`
 * 深層学習の言葉で言えばepoch数が1ということ。
 * ほとんどの状況で、学習が中途半端に終わってしまう。
 * [「gensim lda トピック」でググって](https://www.google.com/search?q=gensim+lda+%E3%83%88%E3%83%94%E3%83%83%E3%82%AF)見つかるほとんどの記事でpassesを指定していない。
 * つまり、gensimを使っているLDAの日本語解説記事の多くが、LDAの本来の性能を示せていない。
* 他の理由: perplexityを底2の対数で求めている。
 * 多くの論文のconventionに反するので、gensimの出力を論文の値と比較できない。

### LDAのモデル構成
* LDAは、テキスト集合から、$K$個のトピックを抽出する。
* 各トピックは、$W$個の語彙の上に定義された確率分布として得られる。
 * 各トピックについて、全語彙にわたって和をとると1になる数値の集まりが得られる。
 * $\phi_k = \{ \phi_{k,1}, \ldots, \phi_{k,W} \}$ s.t. $\sum_{w=1}^W \phi_{k,w} = 1$ for $k=1, \ldots, K$
* LDAを使うと、各テキストにおけるトピックの混合率も分かる。
 * 各テキストについて、全てのトピックにわたって和を求めると1になる数値の集まりが得られる。
 * $\theta_d = \{ \theta_{d,1}, \ldots, \theta_{d,K} \}$ s.t. $\sum_{k=1}^K \theta_{d,k} = 1$ for each document $d$
* 今回は、各トピックにおいて確率の高い単語を、ワードクラウドで可視化する。

## 準備

### spaCy日本語モデルのインストール

In [ ]:
!python -m spacy download ja_core_news_sm

### 日本語フォントのインストール

In [ ]:
!apt-get -y install fonts-ipafont-gothic

### 可視化ツールのインストール

In [ ]:
!pip install pyLDAvis

**ランタイムを再起動する。**

## データセット
* liverdoorニュースコーパスを使う。

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

### 前処理

In [ ]:
import re
import tarfile

tar_fname = "ldcc-20140209.tar.gz"

def read_title(f):
  next(f) # URL
  next(f) # タイムスタンプ
  title = next(f) # 3行目を返す：タイトル
  title = title.decode('utf-8')
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  return re.sub(brackets_head, "", re.sub(brackets_tail, "", title))[:-1]

corpus = []
with tarfile.open(tar_fname) as tf:
  for item in tf:
    if "LICENSE.txt" in item.name:
      continue
    if len(item.name.split('/')) < 3:
      continue
    if not item.name.endswith(".txt"):
      continue
    fname = item.name
    # 今回はクラス名は要らない
    #class_name = fname.split('/')[1]
    f = tf.extractfile(fname)
    title = read_title(f)
    corpus.append(title)

In [ ]:
len(corpus)

* spaCyで形態素解析し、活用語は原形に戻す。
 * 今回は、名詞、固有名詞、動詞、形容詞、副詞のみを残す。

In [ ]:
import spacy
from tqdm import tqdm

nlp = spacy.load("ja_core_news_sm")

pos_list = ["NOUN", "PROPN", "VERB", "ADJ", "ADV"]

lemmatized = []
for text in tqdm(corpus):
  words = [token.lemma_ for token in nlp(text) if token.pos_ in pos_list]
  lemmatized.append(' '.join(words))

In [ ]:
lemmatized[:20]

* 前処理したコーパスを保存しておく。

### 前処理後のコーパスの保存

In [ ]:
import os

#save_path = "./"
save_path = "/content/drive/MyDrive/2023courses/nlp/"

with open(os.path.join(save_path, "lemmatized_livedoor_corpus.txt"), "w") as f:
  for text in lemmatized:
    f.write(f"{text}\n")

### データ行列の作成

* LDAの場合、単語の出現頻度をそのまま使って各文書をベクトル化する。

In [ ]:
import os

save_path = "/content/drive/MyDrive/2023courses/nlp/"

with open(os.path.join(save_path, "lemmatized_livedoor_corpus.txt"), "r") as f:
  lines = f.readlines()
corpus = [line.strip() for line in lines]

* 訓練用コーパスと検証用コーパスに分割する。

In [ ]:
from sklearn.model_selection import train_test_split

train_corpus, valid_corpus = train_test_split(corpus, test_size=0.1, random_state=12345)

* データ行列を作成する。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5, max_df=0.2)
X_train = vectorizer.fit_transform(train_corpus).toarray()
n_samples, n_features = X_train.shape
print(f"training corpus size: {n_samples}, vocabulary size: {n_features}")
X_valid = vectorizer.transform(valid_corpus).toarray()
print(f"validation corpus size: {X_valid.shape[0]}")

### 訓練データ全体のword cloud

* 描画のための準備

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

%config InlineBackend.figure_format = 'retina'

* 訓練データ全体での各単語の出現頻度を求める。

In [ ]:
vocabulary = vectorizer.get_feature_names_out()
word_freq = list(zip(vocabulary, X_train.sum(axis=0)))
word_freq = sorted(word_freq, key=lambda x: - x[1])
print(word_freq[:20])

* word cloudを描画する。

In [ ]:
wordcloud = WordCloud(
    font_path="/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf",
    background_color="white",
    width=1600,
    height=900,
    )
wordcloud.generate_from_frequencies(dict(word_freq))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("word_cloud.png")

* しかし・・・
 * テキスト集合に対して、たった一つword cloudを作ったところで、何が分かるのか？

## LDAによるEDA
* LDAを使うと、一つのコーパスから複数のword cloudを作ることができる。

### LDAのtraining
* 内部的には、変分推論で事後分布のパラメータを推定している。
 * `learning_method="online"`とすることを推奨。
* 抽出するトピックの個数は`n_components`で指定する。
 * これがword cloudの個数になる。

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 20

model = LatentDirichletAllocation(
    n_components=n_components,
    learning_method="online",
    max_iter=20, #学習がおおよそ収束する値に設定
    random_state=12345,
    evaluate_every=1, #何epochごとにモデルを評価するか
    verbose=1, #学習の進行状況をチェック
    )

In [ ]:
model.fit(X_train)

### perplexityメソッドで評価

In [ ]:
model.perplexity(X_train)

In [ ]:
model.perplexity(X_valid)

* scikit-learnのLDAのperplexityメソッドは、なんだかおかしい。

## ハイパーパラメータのチューニング

### perplexityを求める関数
* 各テキストでのトピック確率と、
* 各トピックでの単語確率から、
* 各テキストでの単語確率を求め、
* これをもとにしてコーパスの尤度を求める、
* ・・・というアプローチでperplexityを計算する。

In [ ]:
import numpy as np

def perplexity(model, X):
  components = model.components_
  topic_word_prob = components / components.sum(-1)[:, np.newaxis]
  doc_topic_prob = model.transform(X)
  doc_word_prob = np.dot(doc_topic_prob, topic_word_prob)
  return np.exp(- (X * np.log(doc_word_prob)).sum() / X.sum())

### LDAのtrainingのヘルパ関数

In [ ]:
def lda_train(n_components, max_iter=None, doc_topic_prior=None, topic_word_prior=None):
  if max_iter is None:
    max_iter = 20
  model = LatentDirichletAllocation(
      n_components=n_components,
      doc_topic_prior=doc_topic_prior,
      topic_word_prior=topic_word_prior,
      learning_method="online",
      random_state=12345,
      )
  for epoch in range(max_iter):
    print(f"### epoch {epoch + 1} | ", end="")
    model.partial_fit(X_train)
    print(f"training perp {perplexity(model, X_train):.3f} | ", end="")
    print(f"validation perp {perplexity(model, X_valid):.3f}")
  return model

### ハイパーパラメータのチューニング

* 自分で決めたトピック数に合わせて・・・
* `doc_topic_prior`と`topic_word_prior`をチューニングする。

In [ ]:
n_components = 20
print(f"n_components : {n_components:d}")

for doc_topic_prior in [1e-10, 1e-9, 1e-8]:
  for topic_word_prior in [0.3, 0.5, 0.7]:
    print(f"doc_topic_prior : {doc_topic_prior:.1e} , ", end="")
    print(f"topic_word_prior : {topic_word_prior:.1e}")
    model = lda_train(
        n_components,
        max_iter=10,
        doc_topic_prior=doc_topic_prior,
        topic_word_prior=topic_word_prior,
        )

* 最適な`doc_topic_prior`の値が小さいのは、おそらく、テキストが短いため。

* word cloudの数がたくさんになっても構わないなら・・・
* トピック数をチューニングしてもよい。

In [ ]:
for topic_word_prior in [0.3, 0.5, 0.7]:
  for n_components in [200, 150, 100]:
    print(f"n_components : {n_components:d} , ", end="")
    print(f"topic_word_prior : {topic_word_prior:.1e}")
    model = lda_train(
        n_components,
        max_iter=10,
        doc_topic_prior=1e-10,
        topic_word_prior=topic_word_prior,
        )

### 高確率語をワードクラウドで可視化

In [ ]:
model = LatentDirichletAllocation(
    n_components=20,
    doc_topic_prior=1e-10,
    topic_word_prior=0.5,
    learning_method="online",
    random_state=12345,
    )
X = vectorizer.transform(corpus).toarray()
for epoch in range(50):
  print(f"### epoch {epoch + 1} | ", end="")
  model.partial_fit(X)
  print(f"perplexity {perplexity(model, X):.3f}")

In [ ]:
wordcloud = WordCloud(
    font_path="/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf",
    background_color="white",
    width=1600,
    height=900,
    )

In [ ]:
n_cols = 4

fig, axes = plt.subplots(
    n_components // n_cols ,
    n_cols,
    figsize=(16, 16),
    sharex=True,
    sharey=True,
    )

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  # キーが単語で値が重みの辞書を作っている
  wordcloud.generate_from_frequencies(
      dict(zip(vocabulary, model.components_[i]))
      )
  plt.gca().imshow(wordcloud)
  plt.gca().set_title(f"Topic {i:02d}")
  plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

## pyLDAvisによる可視化

In [ ]:
import pyLDAvis

components = model.components_
components = components / components.sum(-1)[:, np.newaxis]

vis = pyLDAvis.prepare(
  components,
  model.transform(X),
  doc_lengths=X.sum(axis=1),
  vocab=vectorizer.get_feature_names_out(),
  term_frequency=X.sum(axis=0),
  #mds="tsne",
)
pyLDAvis.display(vis)